In [1]:
"""

Selon le document MTH3302_CriteresProjet.pdf nous sommes evalues par notre root mean squared error (RMSE).
On trouve ici son implementation.

Arguments :
    Y : La variable d'interet
    W : Notre estimation de la variable d'interet
    
    
Retourne:
    rmse : L'erreur moyenne de nos predictions

"""

def RMSE(Y,W):

    n = len(Y)

    total = 0.0
    for i in range(n):
        total += (Y[i] - W[i])**2
        
    rmse = total/float(n)
        
    return rmse

In [2]:
"""
--- Variable Qualitative -> Table Binaire --- 

Afin de traiter nos variables qualitatives, nous avons une fonction qui la tranforme en table binaire 
avec un one bit encoder. 

    Disons que la variable explicative peut prendre 4 valeurs {Bleu, Vert, Jaune, Rouge},
        
          ID        X1
        ----------------
          1        Bleu
          2        Bleu
          3        Vert
          4        Jaune
          5        Vert
          6        Rouge          
         ...
    
    Suite a un encodage en 1 bit notre variable explicative X1 devient,
    
          ID        Bleu        Vert        Jaune        Rouge
        -------------------------------------------------------
          1          1           0            0            0
          2          1           0            0            0
          3          0           1            0            0
          4          0           0            1            0
          5          0           1            0            0
          6          0           0            0            1
                                ...
                                

Nous nous basons sur la fonction get_dummies() de la librairie Panda pour faire ceci.


Arguments,

    trainSet : le jeu de donnees de training
    testSet : le jeu de donnees de test
    varName : la variable qualitative que nous souhaitons transformer en tableau binaire
    
    
Retourne, 
    trainSet_new : le jeu de donnees de training ou la variable qualitative a ete remplacee par un tableau binaire
    testSet_new : le jeu de donnees de test ou la variable qualitative a ete remplacee par un tableau binaire

"""

def qualiToBinary(trainSet, testSet, varName):
    
    # Genere la table binaire de la variable qualitative
    trainVar_binary = pd.get_dummies(trainSet[varName],prefix=varName)
    testVar_binary = pd.get_dummies(testSet[varName],prefix=varName)
    
    # On convertit nos tableaux binaires en Dataframe
    trainVar_binary = pd.DataFrame(trainVar_binary)
    testVar_binary = pd.DataFrame(testVar_binary)
    
    # On enleve la variable qualitative de nos jeux de donnees
    trainSet_new = trainSet.drop([varName], axis=1)
    testSet_new = testSet.drop([varName], axis=1)   

    # On ajoute les nouvelles colonnes a nos jeux de donnees
    trainSet_new = pd.concat([trainSet_new,trainVar_binary],axis=1)
    testSet_new = pd.concat([testSet_new,testVar_binary],axis=1)
    
    # On identifie les colonnes manquantes dans le jeu de donnees test
    missing_categories = set(trainSet_new) - set(testSet_new)

    # On ajoute ces colonnes manquantes avec une valeur par defaut de 0
    for c in missing_categories:
        testSet_new[c] = 0

    # On identifie les colonnes manquantes dans le jeu de donnees training
    missing_categories = set(testSet_new) - set(trainSet_new)

    # On ajoute ces colonnes manquantes avec une valeur par defaut de 0
    for c in missing_categories:
        trainSet_new[c] = 0


    # On aligne nos jeux de donnees pour que les memes colonnes soient aux memes endroits
    trainSet_new, testSet_new = trainSet_new.align(testSet_new, axis=1)
    
    # On retourne les jeux de donnees
    return trainSet_new, testSet_new

In [3]:
# On importe les librairies
import numpy as np # calcul matriciel
import pandas as pd # structure des donnees

# Chemin vers les jeux de donnees en format csv
pathTest = "./test3.csv"
pathTrain = "./train3.csv"

# On importe les jeux de donnees
df_test = pd.read_csv(pathTest, sep=',', index_col=0)
df_train = pd.read_csv(pathTrain, sep=',', index_col=0)

# On enleve la colonne Name qui ne sert pas a l'analyse
df_train = df_train.drop(['Name'],axis=1)

In [4]:
"""

Ici on transforme nos variables qualitatives en tableau binaire

"""

# Variable Qualitative #1 : Platform
print("Nbr. of categories Platform (train) : %d" %(df_train.Platform.nunique()))
print("Nbr. of categories Platform (test) : %d \n" %(df_test.Platform.nunique()))
df_train, df_test = qualiToBinary(df_train, df_test, 'Platform') # On binairise


# Variable Qualitative #2 : Genre
print("Nbr. of categories Genre (train) : %d" %(df_train.Genre.nunique()))
print("Nbr. of categories Genre (test) : %d \n" %(df_test.Genre.nunique()))
df_train, df_test = qualiToBinary(df_train, df_test,'Genre') # On binairise


# Variable Qualitative #3 : Publisher
print("Nbr. of categories Publisher (train) : %d" %(df_train.Publisher.nunique()))
print("Nbr. of categories Publisher (test) : %d \n" %(df_test.Publisher.nunique()))
df_train, df_test = qualiToBinary(df_train, df_test, 'Publisher') # On binairise
#df_train = df_train.drop(['Publisher'], axis=1)
#df_test = df_test.drop(['Publisher'], axis=1)

# Variable Qualitative #4 : Developer
print("Nbr. of categories Developer (train) : %d" %(df_train.Developer.nunique()))
print("Nbr. of categories Developer (test) : %d \n" %(df_test.Developer.nunique()))
#df_train, df_test = qualiToBinary(df_train, df_test,'Developer') # On binairise
df_train = df_train.drop(['Developer'], axis=1)
df_test = df_test.drop(['Developer'], axis=1)

# Variable Qualitative #5 : Rating
print("Nbr. of categories Rating (train) : %d" %(df_train.Rating.nunique()))
print("Nbr. of categories Rating (test) : %d \n" %(df_test.Rating.nunique()))
df_train, df_test = qualiToBinary(df_train, df_test,'Rating') # On binairise

Nbr. of categories Platform (train) : 30
Nbr. of categories Platform (test) : 27 

Nbr. of categories Genre (train) : 12
Nbr. of categories Genre (test) : 12 

Nbr. of categories Publisher (train) : 552
Nbr. of categories Publisher (test) : 280 

Nbr. of categories Developer (train) : 1593
Nbr. of categories Developer (test) : 677 

Nbr. of categories Rating (train) : 8
Nbr. of categories Rating (test) : 5 



In [5]:
df_train = df_train.fillna(df_train.median())
df_test = df_test.fillna(df_train.median())

In [6]:
# On fait une matrice de nos variables
X_train = df_train.drop(['NA_Sales','Global_Sales'],axis=1).values
X_test = df_test.drop(['NA_Sales','Global_Sales'],axis=1).values

# On isole nos variables d'interet
Y1 = df_train.Global_Sales.values
Y2 = df_train.NA_Sales.values

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf1 = RandomForestRegressor(n_estimators = 1000, random_state = 42, n_jobs=-1)

# Train the model on training data
rf1.fit(X, Y1);

In [ ]:
# Instantiate model with 1000 decision trees
rf2 = RandomForestRegressor(n_estimators = 300, random_state = 42)

# Train the model on training data
rf2.fit(X_train, Y2);

In [ ]:
# Use the forest's predict method on the test data
Y1_test = rf1.predict(X_test)

# Use the forest's predict method on the test data
Y2_test = rf2.predict(X_test)

In [ ]:
# On vient prendre le dataframe comme canva
df_test_estimated = pd.DataFrame([df_test.NA_Sales,df_test.Global_Sales]).copy().T

# On assigne nos predictions aux colonnes
df_test_estimated['Global_Sales'] = Y1_test
df_test_estimated['NA_Sales'] = Y2_test

# On sauvegarde nos predictions
df_test_estimated.to_csv("test_final.csv")